# Llama2
https://huggingface.co/blog/llama2

In [3]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [1]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-hf" #https://huggingface.co/meta-llama

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)


c:\Users\lkk68\.conda\envs\mycondapy39\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lkk68\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'


In [2]:
#Option1: use pipeline
sequences = pipeline(
    'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?
Thanks for your reply. I'll check out "Spartans".
I also liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?
Yes - you should definitely watch Homeland. It's fantastic. I just finished watching it for a third time.
I also just got started on House of Cards. I've heard lots of great things about it, and it's very good so far. I've only started episode one but I've got to know Kevin Spacey well from watching the Usual Suspects and American Beauty.
I'll check those out too.
I just binge watched "House of Cards" in the last couple of days. It was really good.


In [1]:
#Option2: use model.generate
import torch
from transformers import AutoTokenizer
import transformers
from transformers import AutoModelForCausalLM
model_name = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             torch_dtype=torch.float16,
                                             device_map="auto",
                                             revision="main")

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained(model_name)

In [3]:
import os
mycache_dir="/Users/kaikailiu/Documents/Huggingface"
newpath=os.path.join(mycache_dir, model_name)
os.makedirs(newpath, exist_ok=True)
tokenizer.save_pretrained(newpath)
config.save_pretrained(newpath)
model.save_pretrained(newpath)
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_

In [6]:
device='mps'
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_

In [7]:
prompt = "Tell me about AI"
prompt_template=f'''{prompt}

'''

print("\n\n*** Generate:")
input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids
input_ids.to(device)
output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)
print(tokenizer.decode(output[0]))



*** Generate:


/Users/kaikailiu/miniconda3/envs/mypy310/lib/python3.10/site-packages/transformers/generation/utils.py:1591: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on mps. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('mps') before running `.generate()`.
  warnings.warn(
loc("cast"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/75428952-3aa4-11ee-8b65-46d450270006/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":745:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x1x1x390xi1>'
loc("cast"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/75428952-

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:9                                                                                    │
│                                                                                                  │
│    6 print("\n\n*** Generate:")                                                                  │
│    7 input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids                       │
│    8 input_ids.to(device)                                                                        │
│ ❱  9 output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)              │
│   10 print(tokenizer.decode(output[0]))                                                          │
│   11                                                                                             │
│                                                                                                  │
│ /Users/kaikailiu/miniconda3/envs/mypy310/lib/python3.10/site-packages/torch/utils/_contextlib.py │
│ :115 in decorate_context                                                                         │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /Users/kaikailiu/miniconda3/envs/mypy310/lib/python3.10/site-packages/transformers/generation/ut │
│ ils.py:1719 in generate                                                                          │
│                                                                                                  │
│   1716 │   │   │   )                                                                             │
│   1717 │   │   │                                                                                 │
│   1718 │   │   │   # 13. run sample                                                              │
│ ❱ 1719 │   │   │   return self.sample(                                                           │
│   1720 │   │   │   │   input_ids,                                                                │
│   1721 │   │   │   │   logits_processor=logits_processor,                                        │
│   1722 │   │   │   │   logits_warper=logits_warper,                                              │
│                                                                                                  │
│ /Users/kaikailiu/miniconda3/envs/mypy310/lib/python3.10/site-packages/transformers/generation/ut │
│ ils.py:2801 in sample                                                                            │
│                                                                                                  │
│   2798 │   │   │   model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)  │
│   2799 │   │   │                                                                                 │
│   2800 │   │   │   # forward pass to get next token                                              │
│ ❱ 2801 │   │   │   outputs = self(                                                               │
│   2802 │   │   │   │   **model_inputs,                     

In [1]:
import torch 
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.utils import logging

In [2]:
# Disable warnings about padding_side that cannot be rectified with current software:
logging.set_verbosity_error()

model_names = ["microsoft/DialoGPT-small", "microsoft/DialoGPT-medium", "microsoft/DialoGPT-large"]
use_model_index = 2  # Change 0: small model, 1: medium, 2: large model (requires most resources!)
model_name = model_names[use_model_index]
          
tokenizer = AutoTokenizer.from_pretrained(model_name) # , padding_side='left')
model = AutoModelForCausalLM.from_pretrained(model_name)

In [3]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained(model_name)

In [4]:
import os
mycache_dir="/Users/kaikailiu/Documents/Huggingface"
newpath=os.path.join(mycache_dir, model_name)
os.makedirs(newpath, exist_ok=True)
tokenizer.save_pretrained(newpath)
config.save_pretrained(newpath)
model.save_pretrained(newpath)
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1280)
    (wpe): Embedding(1024, 1280)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-35): 36 x GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1280, out_features=50257, bias=False)
)


In [5]:
# The chat function: received a user input and chat-history and returns the model's reply and chat-history:
def reply(input_text, history=None):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input_text + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([history, new_user_input_ids], dim=-1) if history is not None else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    return tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True), chat_history_ids

In [6]:
history = None
while True:
    input_text = input("> ")
    if input_text in ["", "bye", "quit", "exit"]:
        break
    reply_text, history_new = reply(input_text, history)
    history=history_new
    if history.shape[1]>80:
        old_shape = history.shape
        history = history[:,-80:]
        print(f"History cut from {old_shape} to {history.shape}")
    # history_text = tokenizer.decode(history[0])
    # print(f"Current history: {history_text}")
    print(f"D_GPT: {reply_text}")

D_GPT: I'm not sure what you mean.
D_GPT: I'm not sure what you mean.
D_GPT: I want to know more about AI
D_GPT: I want to know more about AI
D_GPT: I want to know more about AI
History cut from torch.Size([1, 87]) to torch.Size([1, 80])
D_GPT: I want to know more about AI
